# Model Sistem Rekomendasi Film

## Penjelasan Projek

## Work Flow

# Program

## Import Library

In [1]:
#untuk data set
import pandas as pd
import numpy as np

#untuk pencarian kata typo yang diketikan user yang dicocokan kedalam dataset
import difflib

#feature extraction (mengubah jadi angka)
from sklearn.feature_extraction.text import TfidfVectorizer

#Menghitungang kesamaan skor kata dengan yang dicocokan
from sklearn.metrics.pairwise import cosine_similarity

#set tampilan
pd.set_option("Display.max_column",None)

#ignore warning
import warnings
warnings.filterwarnings("ignore")

## Loading Dataset

In [2]:
data_film = pd.read_csv("Dataset/movies.csv")
data_film

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honey

## Seputar Dataset

In [3]:
data_film.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [4]:
data_film.isna().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

## Seleksi Fitur

In [5]:
seleksi_fitur = ["genres","keywords","tagline","cast","director"]
print(seleksi_fitur)

['genres', 'keywords', 'tagline', 'cast', 'director']


## Data treatment

### Data Cleansing

In [6]:
# Mengisi mising value pada seleksi_fitur dengan " " menggunakan perulangan agar data bersih

for fitur in seleksi_fitur:
    data_film[fitur]= data_film[fitur].fillna(" ")

In [7]:
data_film.isna().sum()

index                      0
budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                    0
title                      0
vote_average               0
vote_count                 0
cast                       0
crew                       0
director                   0
dtype: int64

### Penggabungan kolom seleksi fitur

In [8]:
data_film["gabungan"] = data_film["genres"]+" "+data_film["keywords"]+" "+data_film["tagline"]+" "+data_film["cast"]+" "+data_film["director"]

In [10]:
data_film["gabungan"].iloc[1]

"Adventure Fantasy Action ocean drug abuse exotic island east india trading company love of one's life At the end of the world, the adventure begins. Johnny Depp Orlando Bloom Keira Knightley Stellan Skarsg\\u00e5rd Chow Yun-fat Gore Verbinski"

### Stop Word dan Tokenisasi

In [13]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(data_film['gabungan']))
len(norm_corpus)
norm_corpus

array(['action adventure fantasy science fiction culture clash future space war space colony society enter world pandora sam worthington zoe saldana sigourney weaver stephen lang michelle rodriguez james cameron',
       'adventure fantasy action ocean drug abuse exotic island east india trading company love ones life end world adventure begins johnny depp orlando bloom keira knightley stellan skarsgu00e5rd chow yunfat gore verbinski',
       'action adventure crime spy based novel secret agent sequel mi6 plan one escapes daniel craig christoph waltz lu00e9a seydoux ralph fiennes monica bellucci sam mendes',
       ...,
       'comedy drama romance tv movie date love first sight narration investigation team eric mabius kristin booth crystal lowe geoff gustafson benjamin hollingsworth scott smith',
       'new yorker shanghai daniel henney eliza coupe bill paxton alan ruck zhu shimao daniel hsia',
       'documentary obsession camcorder crush dream girl drew barrymore brian herzlinger c

### Konvert text data ke feature vector

In [14]:
#memanggil method Tfidf ke dlam vectorizer
vectorizer = TfidfVectorizer()

In [15]:
#konvert
fitur_vectors = vectorizer.fit_transform(norm_corpus)

In [16]:
print(fitur_vectors)

  (0, 2467)	0.17419810862101923
  (0, 7877)	0.11309109882513116
  (0, 13326)	0.1947312986648133
  (0, 10436)	0.16099617713058897
  (0, 8931)	0.22766899333834206
  (0, 14935)	0.15189290258743376
  (0, 16740)	0.19893842868080433
  (0, 14376)	0.20648588212861022
  (0, 13627)	0.21830207387823347
  (0, 17366)	0.20249395427966974
  (0, 17074)	0.23703591279845068
  (0, 13657)	0.15059552102661705
  (0, 11777)	0.28600862672984995
  (0, 17062)	0.13026399231101185
  (0, 5029)	0.2408709248287723
  (0, 14589)	0.21446706184791187
  (0, 3278)	0.2502378442888809
  (0, 16660)	0.12581419860737125
  (0, 14699)	0.34049320567457786
  (0, 5951)	0.16509483407786396
  (0, 3114)	0.22264985192867034
  (0, 3739)	0.21446706184791187
  (0, 5537)	0.1039055724210519
  (0, 13905)	0.1039055724210519
  (0, 5374)	0.11136877588095698
  :	:
  (4801, 14151)	0.28049432064133684
  (4801, 17343)	0.29046763504571244
  (4801, 4916)	0.24872850542416491
  (4801, 409)	0.1784169981531667
  (4801, 7053)	0.29046763504571244
  (4801, 

### Mendapatakankemirpan score dengan cosine similarity

In [17]:
kemiripan = cosine_similarity(fitur_vectors)

In [18]:
kemiripan

array([[1.        , 0.04851609, 0.03799437, ..., 0.        , 0.        ,
        0.        ],
       [0.04851609, 1.        , 0.02160745, ..., 0.01280076, 0.        ,
        0.        ],
       [0.03799437, 0.02160745, 1.        , ..., 0.        , 0.05447392,
        0.        ],
       ...,
       [0.        , 0.01280076, 0.        , ..., 1.        , 0.        ,
        0.02688131],
       [0.        , 0.        , 0.05447392, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02688131, 0.        ,
        1.        ]])

In [19]:
print(kemiripan.shape)

(4803, 4803)


## Membuat sistem pengambilan judul dari inputan user ( based judul)

In [22]:
#membuat inputan

judul_film = input("Masukan Judul Film yang anda cari : ")

Masukan Judul Film yang anda cari : Iron man


In [23]:
#membuat kolom judul fuilm, menjadi list
list_judul_film_dari_dataset = data_film["title"].tolist()
print(list_judul_film_dari_dataset)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [24]:
# mencari judul film yang paling mirip dengan inputan dari user

pencarian_judul_terdekat_dari_user = difflib.get_close_matches(judul_film, list_judul_film_dari_dataset)
print(pencarian_judul_terdekat_dari_user)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [25]:
#Mengambil judul film dalam list yang paling mirip dengaqn inputan user
judul_paling_mirip = pencarian_judul_terdekat_dari_user[0]
judul_paling_mirip

'Iron Man'

In [26]:
#mencari index dari judul_paling_mirip
index_dari_judul_film = data_film[data_film.title==judul_paling_mirip]['index'].values[0]
print(index_dari_judul_film)

68


In [27]:
# mendaptkan list skor kermiripan dari index judul film yang dicari

kemiripan_skor = list(enumerate(kemiripan[index_dari_judul_film]))
print(kemiripan_skor)

[(0, 0.03525433640851201), (1, 0.06140458665162923), (2, 0.014451438564358328), (3, 0.006816323629226923), (4, 0.03585677602704003), (5, 0.01451786840419423), (6, 0.0369261677725813), (7, 0.2548420795318238), (8, 0.008290020669546929), (9, 0.08152030441708374), (10, 0.079505770059102), (11, 0.013916282870227063), (12, 0.014763067117327666), (13, 0.013239611934453895), (14, 0.10606206119928177), (15, 0.007914146208251184), (16, 0.24195632082773152), (17, 0.014253984109285448), (18, 0.026755234563897397), (19, 0.012571305916104161), (20, 0.08373211946455268), (21, 0.012397650209505099), (22, 0.007328110625643378), (23, 0.0073712619445214875), (24, 0.01343550412353356), (25, 0.0), (26, 0.2294808783488993), (27, 0.03273800557745864), (28, 0.06442433335732738), (29, 0.014954171097120564), (30, 0.08761747712602787), (31, 0.3384257988760995), (32, 0.00780133087277424), (33, 0.1377059332861199), (34, 0.0), (35, 0.0372279159070634), (36, 0.03442592233993875), (37, 0.008670122779513335), (38, 0.

In [28]:
len(kemiripan_skor)

4803

In [29]:
# mengurutkan film berdasarkan skor kemiripan dari yang dicari user

urutan_kemiripan_film = sorted(kemiripan_skor, key = lambda x:x[1], reverse = True) 
print(urutan_kemiripan_film)

[(68, 1.0000000000000002), (79, 0.45987558072021417), (31, 0.3384257988760995), (7, 0.2548420795318238), (16, 0.24195632082773152), (26, 0.2294808783488993), (85, 0.21654364911932902), (182, 0.21277218015781585), (511, 0.17596047402588233), (3623, 0.16955326448340818), (203, 0.16630340866771776), (64, 0.1651648378003598), (174, 0.16034163178297955), (101, 0.15849514591168273), (1740, 0.15375381172480904), (4401, 0.15194226104980663), (169, 0.1461492644448833), (94, 0.1440076136727558), (126, 0.1422322978487096), (131, 0.14172715194104651), (33, 0.1377059332861199), (788, 0.1365270375201086), (46, 0.13511737000801616), (2487, 0.1300163540134726), (783, 0.1293227291476225), (2442, 0.12835120908235334), (138, 0.12402240985684189), (661, 0.12342679378217965), (607, 0.12187606060715546), (38, 0.12185092055478519), (2651, 0.12030090806956666), (122, 0.11396073507302178), (1553, 0.11198683177694307), (242, 0.11189411730461934), (2235, 0.10687851349896837), (2390, 0.10647441284121179), (14, 0.

In [30]:
# pmenceteak rekomendasi film

print('Film yang anda cari : \n')

i = 1

for film in urutan_kemiripan_film:
  index = film[0]
  judul_dari_index = data_film[data_film.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',judul_dari_index)
    i+=1

Film yang anda cari : 

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . Avengers: Age of Ultron
5 . The Avengers
6 . Captain America: Civil War
7 . Captain America: The Winter Soldier
8 . Ant-Man
9 . X-Men
10 . Made
11 . X2
12 . X-Men: Apocalypse
13 . The Incredible Hulk
14 . X-Men: First Class
15 . Kick-Ass 2
16 . The Helix... Loaded
17 . Captain America: The First Avenger
18 . Guardians of the Galaxy
19 . Thor: The Dark World
20 . G-Force
21 . X-Men: The Last Stand
22 . Deadpool
23 . X-Men: Days of Future Past
24 . Duets
25 . Mortdecai
26 . Southland Tales
27 . The Last Airbender
28 . Zathura: A Space Adventure
29 . Sky Captain and the World of Tomorrow


## Sistem Rekomendasi

In [31]:
judul_film = input("Masukan Judul Film yang anda cari : ")

list_judul_film_dari_dataset = data_film["title"].tolist()

pencarian_judul_terdekat_dari_user = difflib.get_close_matches(judul_film, list_judul_film_dari_dataset)

judul_paling_mirip = pencarian_judul_terdekat_dari_user[0]

index_dari_judul_film = data_film[data_film.title==judul_paling_mirip]['index'].values[0]

kemiripan_skor = list(enumerate(kemiripan[index_dari_judul_film]))

urutan_kemiripan_film = sorted(kemiripan_skor, key = lambda x:x[1], reverse = True) 

print('Film yang anda cari : \n')

i = 1

for film in urutan_kemiripan_film:
  index = film[0]
  judul_dari_index = data_film[data_film.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',judul_dari_index)
    i+=1

Masukan Judul Film yang anda cari : X-men
Film yang anda cari : 

1 . X-Men
2 . X2
3 . X-Men: The Last Stand
4 . X-Men: Days of Future Past
5 . X-Men: Apocalypse
6 . The Wolverine
7 . Avengers: Age of Ultron
8 . Ant-Man
9 . The Avengers
10 . Iron Man 2
11 . X-Men Origins: Wolverine
12 . The Incredible Hulk
13 . The Image Revolution
14 . Man of Steel
15 . Thor: The Dark World
16 . X-Men: First Class
17 . The Amazing Spider-Man 2
18 . Deadpool
19 . Captain America: The Winter Soldier
20 . Superman II
21 . Captain America: Civil War
22 . Superman Returns
23 . Flushed Away
24 . Iron Man
25 . Swordfish
26 . Kick-Ass 2
27 . Batman v Superman: Dawn of Justice
28 . Watchmen
29 . The Shadow
